<a href="https://colab.research.google.com/github/Alan-Marcus/HCCS/blob/main/03%20Analysis%20scripts%20and%20templates/SLEAP/Colab%20Notebooks/SLEAP_Scale_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**HCCS: Scale Calculation for SLEAP**

This is an example notebook to calculate the real-world scale for videos recorded using the Home-Cage Camera System (HCCS; https://github.com/Alan-Marcus/HCCS) and using SLEAP (https://sleap.ai) for analysis.
This notebook will extract the coordinates from an H5 analysis file and save them in a CSV file for input into the analysis template.

#0. Setup

1. Upload the scale analysis file to the analysis folder in Google Drive.
2. Run the following cells to setup the required utilities and connect your Google Drive to this session.

In [ ]:
#@title Install Utilities
import h5py
import numpy as np
import pandas as pd
from google.colab import files
from google.colab import drive
print('done')

In [ ]:
#@title Connect Google Drive
drive.mount('/content/drive/')

#1. Specify Analysis File and Convert to CSV


In [ ]:
#@title Edit Parameter Names
path_to_SLEAP_analysis_folder = "/content/drive/MyDrive/SLEAP/analysis" #@param {type:"string"}
h5_scale_file = "HCCS_example_project-scale.analysis.h5"  #@param {type:"string"}

#---don't edit below
h5_to_convert = path_to_SLEAP_analysis_folder + "/" + h5_scale_file

#Create CSV file of tracks 
#Open the HDF5 file using h5py
with h5py.File(h5_to_convert, "r") as f:

#Load all the datasets into a dictionary
  csvdata = {k: v[:] for k, v in f.items()}

#Convert string arrays into regular Python strings
  csvdata["node_names"] = [s.decode() for s in csvdata["node_names"].tolist()]
  csvdata["track_names"] = [s.decode() for s in csvdata["track_names"].tolist()]

#Transpose the tracks axes
  csvdata["tracks"] = np.transpose(csvdata["tracks"])

#Convert the data type of the track occupancy array to boolean
#  csvdata["track_occupancy"] = csvdata["track_occupancy"].astype(bool)  #original code - excludes empty tracks
  csvdata["track_occupancy_all"] = csvdata["track_occupancy"] + 1  #Use to include empty tracks in output
  csvdata["track_occupancy_all"] = csvdata["track_occupancy_all"].astype(bool)  #Use to include empty tracks in output

#Describe the values in the data dictionary
for key, value in csvdata.items():
 if isinstance(value, np.ndarray):
   print(f"{key}: {value.dtype} array of shape {value.shape}")
 else:
   print(f"{key}: {value}")

#Array of frames that have at least one animal tracked
#valid_frame_idxs = np.argwhere(csvdata["track_occupancy"].any(axis=1)).flatten()  #original code - excludes empty tracks
valid_frame_idxs = np.argwhere(csvdata["track_occupancy_all"].any(axis=1)).flatten()  #Use to include empty tracks in output
valid_frame_idxs

#Generate a 'tracks' table where each row contains the detected body part coordinates for a single animal in a single frame
tracks = []
for frame_idx in valid_frame_idxs:
#Get the tracking data for the current frame
  frame_tracks = csvdata["tracks"][frame_idx]

#Loop over the animals in the current frame
  for i in range(frame_tracks.shape[-1]):
    pts = frame_tracks[..., i]
    
#Uncomment these lines to skip this animal for the current frame if all of its points are missing (i.e. uncomment to not include empty rows)
    if np.isnan(pts).all():
      continue
    
#Initialize our row with some metadata
    detection = {"track": csvdata["track_names"][i], "frame_idx": frame_idx}

#Fill in the coordinates for each body part
    for node_name, (x, y) in zip(csvdata["node_names"], pts):
      detection[f"{node_name}.x"] = x
      detection[f"{node_name}.y"] = y

#Add the row to the list and move on to the next detection
    tracks.append(detection)

#Convert completed list of rows into a table using Pandas
tracks = pd.DataFrame(tracks)

#Save tracks table as a CSV file
tracks.to_csv(h5_to_convert + "_coordinates.csv", index=False)

#Print
print("File saved to: "+ h5_to_convert + "_coordinates.csv")

#View first 10 rows of tracks table
tracks.head(10)

#2. Download output

In [ ]:
#@title Download the coordinates file
files.download(h5_to_convert + "_coordinates.csv")